In [ ]:
# %pip install --upgrade protobuf==4.25.3
# %pip install --upgrade langchain-google-genai google-generativeai

In [ ]:
# %pip install --upgrade langchain langchain-core langchain-community

In [ ]:
# import google.generativeai as genai
# import langchain
# import langchain_google_genai
# import google

# print(genai.__version__)
# print(langchain.__version__)
# # print(langchain_google_genai.__version__)
# print(google.protobuf.__version__)

In [1]:
# Import necessary libraries
import pandas as pd
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os
import getpass
import json

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # ============================================================
# # 2. CHUNK RISK DEFINITIONS AND CREATE VECTOR STORE
# # ============================================================

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,
#     chunk_overlap=50
# )

# risk_chunks = text_splitter.split_text(risk_definitions_text)

# # use Google GenAI embeddings (class is imported at top of the notebook)
# embeddings = GoogleGenerativeAIEmbeddings()

# # Chroma expects the kwarg name `embeddings`
# vectorstore = Chroma.from_texts(risk_chunks, embeddings=embeddings)

# retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
# ``````python
# # ============================================================
# # 3. BUILD RAG LLM CHAIN
# # ============================================================

# # use Google GenAI chat model (class is imported at top of the notebook)
# llm = ChatGoogleGenerativeAI(model="chat-bison-001", temperature=0)

# prompt = ChatPromptTemplate.from_template("""
# You are a senior banking risk officer.
# Use the retrieved bank-specific risk definitions below to classify the event.

# === Risk Definitions ===
# {context}

# === Event Description ===
# {issue_description}

# TASKS:
# 1. Produce a 5–8 word "gist" capturing the core issue.
# 2. Provide a short "rationale" (1 sentence) explaining *why* the chosen risk applies.
# 3. Select ONLY the single most appropriate "risk_type" using the risk definitions.

# Return ONLY JSON in this format:
# {{
#   "gist": "...",
#   "rationale": "...",
#   "risk_type": "..."
# }}
# """)

# rag_chain = (
#     RunnableParallel(
#         context=retriever,
#         issue_description=RunnablePassthrough()
#     )
#     | prompt
#     | llm
# )
# ``````python
# # ============================================================
# # 4. PROCESS THE ENTIRE DATASET
# # ============================================================

# gists = []
# rationales = []
# risk_types = []

# for desc in df["issue_description"].tolist():
#     out = rag_chain.invoke({"issue_description": desc})

#     # normalize to a string containing the model response
#     if hasattr(out, "content"):
#         raw = out.content
#     elif isinstance(out, (list, tuple)) and len(out) > 0:
#         first = out[0]
#         raw = getattr(first, "content", str(first))
#     else:
#         raw = str(out)

#     parsed = None
#     # try eval (legacy) then JSON
#     try:
#         parsed = eval(raw)
#     except Exception:
#         try:
#             parsed = json.loads(raw)
#         except Exception:
#             parsed = None

#     if isinstance(parsed, dict):
#         gists.append(parsed.get("gist", ""))
#         rationales.append(parsed.get("rationale", ""))
#         risk_types.append(parsed.get("risk_type", ""))
#     else:
#         # fallback
#         gists.append("Unable to extract gist")
#         rationales.append("Unable to infer rationale")
#         risk_types.append("Unknown")

# df["gist"] = gists
# df["rationale"] = rationales
# df["risk_type"] = risk_types

In [ ]:
# import google.generativeai as genai
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
# genai.configure(api_key=GOOGLE_API_KEY)

In [37]:
# Setup Google Generative AI
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API key here ")

In [3]:
# ============================================================
# 1. READ USER-UPLOADED FILES
# ============================================================

csv_path = "./synthetic_risk_incidents_2025.csv"
risk_def_path = "./Non Financial Risk.txt"

df = pd.read_csv(csv_path)

with open(risk_def_path, "r", encoding="utf-8") as f:
    risk_definitions_text = f.read()

In [6]:
# ============================================================
# 2. CHUNK RISK DEFINITIONS AND CREATE VECTOR STORE
# ============================================================

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

risk_chunks = text_splitter.split_text(risk_definitions_text)

# use Google GenAI embeddings (class is imported at top of the notebook)
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Chroma expects the kwarg name `embeddings`
vectorstore = Chroma.from_texts(texts=risk_chunks, embedding=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [34]:
response = retriever.invoke('whats is Regulatory Conduct?')

In [35]:
response

[Document(metadata={}, page_content='Regulatory Conduct Risk: Potential for loss or adverse impact due to failure to adequately address and meet regulatory requirements and expectations in relation to a robust compliance risk culture.\nNon Financial Regulatory Reporting Risk: Potential for loss or adverse impact due to failure to comply with non financial regulatory reporting requirements by submitting inaccurate, incomplete, and/or untimely reports to the relevant regulatory authority (excluding tax reporting obligations)'),
 Document(metadata={}, page_content='Market Conduct Risk: Potential for loss or adverse impact due to the Bank engaging in inappropriate, negligent, or intentional market activity or behaviour that may impact the integrity of financial markets.\nClient Conduct Risk: Potential for loss or adverse impact due to the Bank not achieving fair outcomes for clients when providing products and services.'),
 Document(metadata={}, page_content="Conflict of Interest Risk: Pot

In [38]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
# llm = ChatGoogleGenerativeAI(model="chat-bison-001", temperature=0)

resp = llm.invoke("What is AI Agent in null shell?")
print(resp)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..


NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
# ============================================================
# 3. BUILD RAG LLM CHAIN
# ============================================================

prompt = ChatPromptTemplate.from_template("""
You are a senior banking risk officer.
Use the retrieved bank-specific risk definitions below to classify the event.

=== Risk Definitions ===
{context}

=== Event Description ===
{issue_description}

TASKS:
1. Produce a 5–8 word "gist" capturing the core issue.
2. Provide a short "rationale" (1 sentence) explaining *why* the chosen risk applies.
3. Select ONLY the single most appropriate "risk_type" using the risk definitions.

Return ONLY JSON in this format:
{{
  "gist": "...",
  "rationale": "...",
  "risk_type": "..."
}}
""")

rag_chain = (
    RunnableParallel(
        context=retriever,
        issue_description=RunnablePassthrough()
    )
    | prompt
    | llm
)

In [10]:
df["issue_description"].tolist()

['Failure to adhere to regulatory expectations resulted in a conflict of interest risk event impacting compliance obligations and audit outcomes.',
 'Failure to adhere to regulatory expectations resulted in a client conduct risk event impacting compliance obligations and audit outcomes.',
 'Employee oversight triggered a significant conflict of interest risk incident involving incorrect handling of sensitive data and operational disruption.',
 'Failure to identify conflicting interests early led to a client conduct risk scenario requiring risk committee review.',
 'Failure to identify conflicting interests early led to a client conduct risk scenario requiring risk committee review.',
 'Failure to adhere to regulatory expectations resulted in a regulatory conduct risk event impacting compliance obligations and audit outcomes.',
 'Employee oversight triggered a significant conflict of interest risk incident involving incorrect handling of sensitive data and operational disruption.',
 'Mo

In [13]:
# ============================================================
# 4. PROCESS THE ENTIRE DATASET
# ============================================================

gists = []
rationales = []
risk_types = []

for desc in df["issue_description"].tolist():
    out = rag_chain.invoke(desc)
    try:
        parsed = eval(out.content)
        gists.append(parsed.get("gist", ""))
        rationales.append(parsed.get("rationale", ""))
        risk_types.append(parsed.get("risk_type", ""))
    except:
        # fallback
        gists.append("Unable to extract gist")
        rationales.append("Unable to infer rationale")
        risk_types.append("Unknown")

df["gist"] = gists
df["rationale"] = rationales
df["risk_type"] = risk_types

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/chat-bison-001 is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..


NotFound: 404 models/chat-bison-001 is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
# # ============================================================
# # 4. PROCESS THE ENTIRE DATASET
# # ============================================================

# gists = []
# rationales = []
# risk_types = []

# for desc in df["issue_description"].tolist():
#     out = rag_chain.invoke({"issue_description": desc})

#     # normalize to a string containing the model response
#     if hasattr(out, "content"):
#         raw = out.content
#     elif isinstance(out, (list, tuple)) and len(out) > 0:
#         first = out[0]
#         raw = getattr(first, "content", str(first))
#     else:
#         raw = str(out)

#     parsed = None
#     # try eval (legacy) then JSON
#     try:
#         parsed = eval(raw)
#     except Exception:
#         try:
#             parsed = json.loads(raw)
#         except Exception:
#             parsed = None

#     if isinstance(parsed, dict):
#         gists.append(parsed.get("gist", ""))
#         rationales.append(parsed.get("rationale", ""))
#         risk_types.append(parsed.get("risk_type", ""))
#     else:
#         # fallback
#         gists.append("Unable to extract gist")
#         rationales.append("Unable to infer rationale")
#         risk_types.append("Unknown")

# df["gist"] = gists
# df["rationale"] = rationales
# df["risk_type"] = risk_types

In [ ]:
# ============================================================
# 5. SAVE FINAL DATAFRAME
# ============================================================

output_path = "/mnt/data/risk_classified_output.csv"
df.to_csv(output_path, index=False)

output_path